**Copyright 2023 The TF-Agents Authors.**

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# SAC en Minitaur con la API Actor-Learner

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/agents/tutorials/7_SAC_minitaur_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/7_SAC_minitaur_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/7_SAC_minitaur_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/agents/tutorials/7_SAC_minitaur_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a>
</td>
</table>


## Introducción

Este ejemplo nos muestra cómo entrenar un agente [Soft Actor Critic](https://arxiv.org/abs/1812.05905) en el entorno [Minitaur](https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_envs/bullet/minitaur.py).

Si ha trabajado con [DQN Colab](https://github.com/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb), esto le resultará muy familiar. Algunos cambios importantes son los siguientes:

- Cambio del agente, de DQN a SAC.
- Entrenamiento en Minitaur, que es un entorno mucho más complejo que CartPole. El entorno Minitaur pretende entrenar a un robot cuadrúpedo para que se desplace hacia delante.
- Uso de la API Actor-Learner de TF-Agents para el aprendizaje por refuerzo distribuido.

La API admite tanto la recopilación de datos distribuidos mediante un búfer de repetición de experiencias y un contenedor de variables (servidor de parámetros) como el entrenamiento distribuido en múltiples dispositivos. La API fue diseñada para que sea muy simple y modular. Se usa [Reverb](https://deepmind.com/research/open-source/Reverb) tanto para el búfer de repetición como para el contenedor de variables y [la API DistributionStrategy de TF](https://www.tensorflow.org/guide/distributed_training) para el entrenamiento distribuido en GPU y TPU.

Si todavía no ha instalado las siguientes dependencias, ejecute estos comandos:

In [ ]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg
!pip install 'imageio==2.4.0'
!pip install matplotlib
!pip install tf-agents[reverb]
!pip install pybullet

## Preparación

Primero importaremos las diferentes herramientas que necesitamos.

In [ ]:
import base64
import imageio
import IPython
import matplotlib.pyplot as plt
import os
import reverb
import tempfile
import PIL.Image

import tensorflow as tf

from tf_agents.agents.ddpg import critic_network
from tf_agents.agents.sac import sac_agent
from tf_agents.agents.sac import tanh_normal_projection_network
from tf_agents.environments import suite_pybullet
from tf_agents.metrics import py_metrics
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import greedy_policy
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_py_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.train import actor
from tf_agents.train import learner
from tf_agents.train import triggers
from tf_agents.train.utils import spec_utils
from tf_agents.train.utils import strategy_utils
from tf_agents.train.utils import train_utils

tempdir = tempfile.gettempdir()

## Hiperparámetros

In [ ]:
env_name = "MinitaurBulletEnv-v0" # @param {type:"string"}

# Use "num_iterations = 1e6" for better results (2 hrs)
# 1e5 is just so this doesn't take too long (1 hr)
num_iterations = 100000 # @param {type:"integer"}

initial_collect_steps = 10000 # @param {type:"integer"}
collect_steps_per_iteration = 1 # @param {type:"integer"}
replay_buffer_capacity = 10000 # @param {type:"integer"}

batch_size = 256 # @param {type:"integer"}

critic_learning_rate = 3e-4 # @param {type:"number"}
actor_learning_rate = 3e-4 # @param {type:"number"}
alpha_learning_rate = 3e-4 # @param {type:"number"}
target_update_tau = 0.005 # @param {type:"number"}
target_update_period = 1 # @param {type:"number"}
gamma = 0.99 # @param {type:"number"}
reward_scale_factor = 1.0 # @param {type:"number"}

actor_fc_layer_params = (256, 256)
critic_joint_fc_layer_params = (256, 256)

log_interval = 5000 # @param {type:"integer"}

num_eval_episodes = 20 # @param {type:"integer"}
eval_interval = 10000 # @param {type:"integer"}

policy_save_interval = 5000 # @param {type:"integer"}

## Entorno

Los entornos en RL representan la tarea o problema que tratamos de resolver. Los entornos estándar se pueden crear fácilmente en TF-Agents utilizando `suites`. Hay diferentes `suites` para cargar entornos desde fuentes como OpenAI Gym, Atari, DM Control, etc., dado un nombre de entorno de cadena.

Ahora, carguemos el entorno Minitaur desde el paquete de Pybullet.

In [ ]:
env = suite_pybullet.load(env_name)
env.reset()
PIL.Image.fromarray(env.render())

En este entorno, el objetivo es que el agente entrene una política que controlará el robot Minitaur y hará que avance lo más rápido posible. Los episodios duran 1000 pasos y la devolución será la suma de recompensas a lo largo del episodio.

Miremos la información que proporciona el entorno como una `observation` que la política utilizará para generar `actions`.

In [ ]:
print('Observation Spec:')
print(env.time_step_spec().observation)
print('Action Spec:')
print(env.action_spec())

La observación es bastante compleja. Recibimos 28 valores que representan los ángulos, las velocidades y los pares de todos los motores. A cambio, el entorno espera 8 valores para las acciones entre `[-1, 1]`. Estos son los ángulos del motor deseados.

Por lo general, creamos dos entornos: uno para recopilar datos durante el entrenamiento y otro para evaluación. Los entornos están escritos en Python puro y utilizan arreglos numpy, que la API Actor Learner consume directamente.

In [ ]:
collect_env = suite_pybullet.load(env_name)
eval_env = suite_pybullet.load(env_name)

## Estrategia de distribución

Usamos la API DistributionStrategy para habilitar la ejecución de la computación del paso del entrenamiento en múltiples dispositivos, como en varias GPU o TPU, a través del paralelismo de datos. El paso de entrenamiento:

- Recibe un lote de datos de entrenamiento.
- Lo divide entre los dispositivos.
- Calcula el paso hacia adelante.
- Agrega y calcula la MEDIA de la pérdida.
- Calcula el paso hacia atrás y ejecuta una actualización de la variable del gradiente.

Con la API TF-Agents Learner y la API DistributionStrategy es bastante fácil pasar de ejecución del paso de entrenamiento en GPU (con MirroredStrategy) a TPU (con TPUStrategy) sin cambiar ninguna de las siguientes lógicas de entrenamiento.

### Cómo habilitar la GPU

Si desea probar la ejecución en una GPU, primero deberá habilitar las GPU para el bloc de notas:

- Vaya a Editar → Configuración del bloc de notas
- Seleccione GPU en el menú desplegable Acelerador de hardware

### Cómo elegir una estrategia

Use `strategy_utils` para generar una estrategia. A nivel interno, pase el siguiente parámetro:

- `use_gpu = False` devuelve `tf.distribute.get_strategy()`, que usa CPU
- `use_gpu = True` devuelve `tf.distribute.MirroredStrategy()`, que usa todas las GPU visibles para TensorFlow en una máquina

In [ ]:
use_gpu = True #@param {type:"boolean"}

strategy = strategy_utils.get_strategy(tpu=False, use_gpu=use_gpu)

Todas las variables y agentes deben crearse en `strategy.scope()`, como verá a continuación.

## Agente

Para crear un agente SAC, primero debemos crear las redes que se entrenarán. SAC es un agente actor-crítico, por lo que necesitaremos dos redes.

El crítico nos dará estimaciones de valor para `Q(s,a)`. Es decir, recibirá como entrada una observación y una acción, y nos indicará en qué medida fue buena esa acción para el estado dado.


In [ ]:
observation_spec, action_spec, time_step_spec = (
      spec_utils.get_tensor_specs(collect_env))

with strategy.scope():
  critic_net = critic_network.CriticNetwork(
        (observation_spec, action_spec),
        observation_fc_layer_params=None,
        action_fc_layer_params=None,
        joint_fc_layer_params=critic_joint_fc_layer_params,
        kernel_initializer='glorot_uniform',
        last_kernel_initializer='glorot_uniform')

Usaremos este crítico para entrenar una red `actor` que nos permitirá generar acciones a partir de una observación.

`ActorNetwork` predecirá los parámetros para una distribución tanh-squashed [MultivariateNormalDiag](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/MultivariateNormalDiag). Luego, se tomarán muestras de esta distribución, condicionadas a la observación actual, cada vez que sea necesario generar acciones.

In [ ]:
with strategy.scope():
  actor_net = actor_distribution_network.ActorDistributionNetwork(
      observation_spec,
      action_spec,
      fc_layer_params=actor_fc_layer_params,
      continuous_projection_net=(
          tanh_normal_projection_network.TanhNormalProjectionNetwork))

Ahora, cestas redes a mano, podemos crear una instancia del agente.


In [ ]:
with strategy.scope():
  train_step = train_utils.create_train_step()

  tf_agent = sac_agent.SacAgent(
        time_step_spec,
        action_spec,
        actor_network=actor_net,
        critic_network=critic_net,
        actor_optimizer=tf.keras.optimizers.Adam(
            learning_rate=actor_learning_rate),
        critic_optimizer=tf.keras.optimizers.Adam(
            learning_rate=critic_learning_rate),
        alpha_optimizer=tf.keras.optimizers.Adam(
            learning_rate=alpha_learning_rate),
        target_update_tau=target_update_tau,
        target_update_period=target_update_period,
        td_errors_loss_fn=tf.math.squared_difference,
        gamma=gamma,
        reward_scale_factor=reward_scale_factor,
        train_step_counter=train_step)

  tf_agent.initialize()

## Búfer de repetición

Para hacer un seguimiento de los datos recogidos del entorno, se usará [Reverb](https://deepmind.com/research/open-source/Reverb), un sistema de repetición eficiente, extensible y fácil de usar de Deepmind. Almacena los datos de experiencia recopilados por los Actores y utilizados por el Aprendiz durante el entrenamiento.

Para este tutorial, esto es menos importante que `max_size` -- pero en un entorno distribuido con recopilación y entrenamiento asíncronos, probablemente quiera experimentar con `rate_limiters.SampleToInsertRatio`, usando un samples_per_insert entre 2 y 1000. Por ejemplo:

```
rate_limiter=reverb.rate_limiters.SampleToInsertRatio(samples_per_insert=3.0, min_size_to_sample=3, error_buffer=3.0)
```


In [ ]:
table_name = 'uniform_table'
table = reverb.Table(
    table_name,
    max_size=replay_buffer_capacity,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1))

reverb_server = reverb.Server([table])

El búfer de repetición se construye a partir de especificaciones que describen los tensores que se van a almacenar, y que se pueden obtener del agente si se usa `tf_agent.collect_data_spec`.

Dado que el agente SAC necesita tanto la observación actual como la siguiente para calcular la pérdida, configuramos `sequence_length=2`.

In [ ]:
reverb_replay = reverb_replay_buffer.ReverbReplayBuffer(
    tf_agent.collect_data_spec,
    sequence_length=2,
    table_name=table_name,
    local_server=reverb_server)

Ahora, generamos un conjunto de datos de TensorFlow a partir del búfer de repetición Reverb. Le pasaremos esto al alumno para extraer muestras de experiencias para el entrenamiento.

In [ ]:
dataset = reverb_replay.as_dataset(
      sample_batch_size=batch_size, num_steps=2).prefetch(50)
experience_dataset_fn = lambda: dataset

## Políticas

En TF-Agents, las políticas representan la noción estándar de políticas en RL: dado que un `time_step` produce una acción o una distribución de acciones. El método principal es `policy_step = policy.step(time_step)` donde `policy_step` es una tupla nombrada `PolicyStep(action, state, info)`. `policy_step.action` es la `action` que se aplicará al entorno, `state` representa el estado de las políticas de estado (RNN) y `info` podría contener información auxiliar como probabilidades logarítmicas de las acciones.

Los agentes contienen dos políticas:

- `agent.policy`: la política principal que se usa para la evaluación y la implementación.
- `agent.collect_policy`: una segunda política que se usa para la recopilación de datos.

In [ ]:
tf_eval_policy = tf_agent.policy
eval_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_eval_policy, use_tf_function=True)

In [ ]:
tf_collect_policy = tf_agent.collect_policy
collect_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_collect_policy, use_tf_function=True)

Las políticas pueden crearse independientemente de los agentes. Por ejemplo, utilice `tf_agents.policies.random_py_policy` para crear una política que seleccionará aleatoriamente una acción para cada time_step.

In [ ]:
random_policy = random_py_policy.RandomPyPolicy(
  collect_env.time_step_spec(), collect_env.action_spec())

## Actores

El actor gestiona las interacciones entre una política y un entorno.

- Los componentes del Actor contienen una instancia del entorno (como `py_environment`) y una copia de las variables de la política.
- Cada agente Actor ejecuta una secuencia de pasos de recopilación de datos en función de los valores locales de las variables de la política.
- Las variables se actualizan explícitamente a través de la instancia del cliente contenedor de variables en el script de entrenamiento antes de llamar a `actor.run()`.
- La experiencia observada se escribe en el búfer de repetición en cada paso de recopilación de datos.

A medida que los Actores ejecutan los pasos de recopilación de datos, pasan trayectorias de (estado, acción, recompensa) al observador, que las almacena en caché y las escribe en el sistema de repetición Reverb.

Estamos almacenando trayectorias para marcos [(t0,t1) (t1,t2) (t2,t3), ...] dado que `stride_length=1`.

In [ ]:
rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  reverb_replay.py_client,
  table_name,
  sequence_length=2,
  stride_length=1)

Creamos un Actor con la política aleatoria y recopilamos experiencias para introducirlas en el búfer de repetición.

In [ ]:
initial_collect_actor = actor.Actor(
  collect_env,
  random_policy,
  train_step,
  steps_per_run=initial_collect_steps,
  observers=[rb_observer])
initial_collect_actor.run()

Cree una instancia Actor con la política de recopilación para reunir más experiencias durante el entrenamiento.

In [ ]:
env_step_metric = py_metrics.EnvironmentSteps()
collect_actor = actor.Actor(
  collect_env,
  collect_policy,
  train_step,
  steps_per_run=1,
  metrics=actor.collect_metrics(10),
  summary_dir=os.path.join(tempdir, learner.TRAIN_DIR),
  observers=[rb_observer, env_step_metric])

Cree un Actor que se usará para evaluar la política durante el entrenamiento. Pasamos `actor.eval_metrics(num_eval_episodes)` para registrar las métricas más adelante.

In [ ]:
eval_actor = actor.Actor(
  eval_env,
  eval_policy,
  train_step,
  episodes_per_run=num_eval_episodes,
  metrics=actor.eval_metrics(num_eval_episodes),
  summary_dir=os.path.join(tempdir, 'eval'),
)

## Aprendices

El componente Aprendiz contiene el agente y actualiza gradualmente las variables de la política a partir de los datos de experiencia del búfer de repetición. Después de uno o más pasos de entrenamiento, el Aprendiz puede enviar un nuevo conjunto de valores de variables al contenedor de variables.

In [ ]:
saved_model_dir = os.path.join(tempdir, learner.POLICY_SAVED_MODEL_DIR)

# Triggers to save the agent's policy checkpoints.
learning_triggers = [
    triggers.PolicySavedModelTrigger(
        saved_model_dir,
        tf_agent,
        train_step,
        interval=policy_save_interval),
    triggers.StepPerSecondLogTrigger(train_step, interval=1000),
]

agent_learner = learner.Learner(
  tempdir,
  train_step,
  tf_agent,
  experience_dataset_fn,
  triggers=learning_triggers,
  strategy=strategy)

## Métricas y evaluación

Creamos una instancia del Actor eval con `actor.eval_metrics`, que crea las métricas más utilizadas durante la evaluación de políticas:

- Rendimiento medio. El rendimiento es la suma de las recompensas que se obtienen al ejecutar una política en un entorno durante un episodio, y normalmente se calcula el promedio de varios episodios.
- Duración media de los episodios.

Para generar estas métricas, ejecutamos el Actor.

In [ ]:
def get_eval_metrics():
  eval_actor.run()
  results = {}
  for metric in eval_actor.metrics:
    results[metric.name] = metric.result()
  return results

metrics = get_eval_metrics()

In [ ]:
def log_eval_metrics(step, metrics):
  eval_results = (', ').join(
      '{} = {:.6f}'.format(name, result) for name, result in metrics.items())
  print('step = {0}: {1}'.format(step, eval_results))

log_eval_metrics(0, metrics)

Consulte el [módulo de métricas](https://github.com/tensorflow/agents/blob/master/tf_agents/metrics/tf_metrics.py) para conocer otras implementaciones estándar de diferentes métricas.

## Entrenamiento del agente

El bucle de entrenamiento implica tanto la recopilación de datos del entorno como la optimización de las redes del agente. A lo largo del proceso, evaluaremos de vez en cuando la política del agente para ver nuestro rendimiento.

In [ ]:
#@test {"skip": true}
try:
  %%time
except:
  pass

# Reset the train step
tf_agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = get_eval_metrics()["AverageReturn"]
returns = [avg_return]

for _ in range(num_iterations):
  # Training.
  collect_actor.run()
  loss_info = agent_learner.run(iterations=1)

  # Evaluating.
  step = agent_learner.train_step_numpy

  if eval_interval and step % eval_interval == 0:
    metrics = get_eval_metrics()
    log_eval_metrics(step, metrics)
    returns.append(metrics["AverageReturn"])

  if log_interval and step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, loss_info.loss.numpy()))

rb_observer.close()
reverb_server.stop()

## Visualización


### Gráficos

Podemos hacer una gráfica comparativa del rendimiento medio y los pasos globales para ver el funcionamiento de nuestro agente. En `Minitaur`, la función de recompensa se basa en la distancia que camina el minitaur en 1000 pasos y penaliza el gasto de energía.

In [ ]:
#@test {"skip": true}

steps = range(0, num_iterations + 1, eval_interval)
plt.plot(steps, returns)
plt.ylabel('Average Return')
plt.xlabel('Step')
plt.ylim()

### Videos

Resulta útil visualizar el funcionamiento de un agente mediante el renderizado del entorno en cada paso. Pero antes de hacerlo, creemos una función para insertar videos en este colab.

In [ ]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

El siguiente código visualiza la política del agente para algunos episodios:

In [ ]:
num_episodes = 3
video_filename = 'sac_minitaur.mp4'
with imageio.get_writer(video_filename, fps=60) as video:
  for _ in range(num_episodes):
    time_step = eval_env.reset()
    video.append_data(eval_env.render())
    while not time_step.is_last():
      action_step = eval_actor.policy.action(time_step)
      time_step = eval_env.step(action_step.action)
      video.append_data(eval_env.render())

embed_mp4(video_filename)